#Using Word2Vec и FastText and nearest Neighbors with Annoy

In [1]:
!pip install pymorphy2 stop_words annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 647 kB 19.6 MB/s 
     |████████████████████████████████| 8.2 MB 34.0 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=203405e107ce4733532afc6c1f5adb040264fe874e541b4b214b5eed04688328
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=3a2918054c51fdb915424ab9b52b6695116d25f6a4e7d4a548624091ed569485
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
  Created wheel for annoy: filename=annoy-1.17.1-cp37-cp37m-linux_x86_64.whl size=395176 sha256=eaef7fcff39ac40d742137cea041deb775b0f2e5ff7e43476002d6e0d5339124
  Stored in directory: /root/.cache/pi

In [26]:
import os
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
import random

In [22]:
#загрузка данных с yandex disk
!wget -O train.csv https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/03C9AjRJqukWcg

--2022-10-24 21:01:36--  https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/03C9AjRJqukWcg
Resolving getfile.dokpub.com (getfile.dokpub.com)... 142.132.255.217
Connecting to getfile.dokpub.com (getfile.dokpub.com)|142.132.255.217|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloader.disk.yandex.ru/disk/f402cab5af28ca56cdfd21b0e47d365ea9480e8a0890d5ad4e8586c7981ecc62/6357350e/msMO0KQIAH3VfHbAiTRYc7IU3Nhh3xJ0wfmoVk6qNzxYgqaBMRY0efTvRnMIfEhMPEeBAfXl4uChRAP2dvlWOg%3D%3D?uid=0&filename=train_brands.csv&disposition=attachment&hash=Q%2BL0DOkc2P3%2BfmzFCyxRcXa2Pi%2B4abC8b81eIqanwFGKgEtQzyjIoJ0TbqC7RIGwq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=text%2Fplain&owner_uid=6709741&fsize=29759004&hid=d3075775f2ccb8067d5c2aa80ed316de&media_type=spreadsheet&tknv=v2 [following]
--2022-10-24 21:01:37--  https://downloader.disk.yandex.ru/disk/f402cab5af28ca56cdfd21b0e47d365ea9480e8a0890d5ad4e8586c7981ecc62/6357350e/msMO0KQIAH3VfHbAiTRYc7IU3N

In [23]:
train_path = 'train.csv'

In [24]:
train_df = pd.read_csv(train_path)
train_df

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0
...,...,...,...,...
497814,497815,BIT-MAT PRODUCTS,The Goodyear Tire and Rubber Company,0
497815,497816,"Bnd Trading Co., Ltd.",Zhong Shan Yue Liang Economy& Trade Imp. & Exp...,0
497816,497817,"Xeikon Industrial Co., Ltd. Of Dongguan City","Yi Cheng Trading Co., Ltd. Of Dongguan City",0
497817,497818,"Shanghai Kechuan Trading Co., Ltd.",Shanghai M&G Stationery Inc.,0


In [5]:
train_df.is_duplicate.value_counts()

0    494161
1      3658
Name: is_duplicate, dtype: int64

In [6]:
train_df.is_duplicate.value_counts(normalize=True)*100

0    99.265195
1     0.734805
Name: is_duplicate, dtype: float64

In [ ]:
train_df[train_df.is_duplicate==1][:20]

,pair_id,name_1,name_2,is_duplicate
161,162,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Technical Serv...,1
603,604,Pirelli Neumaticos S.A.I.C.,"Pirelli Tyre Co., Ltd.",1
835,836,Brenntag Australia (Pty) Ltd.,Brenntag Group,1
1328,1329,"PAUL BAUDER GMBH & CO KG, BOCHUM PLANT",Paul Bauder ag,1
1562,1563,TOTAL CESKA REPUBLIKA s.r.o.,TOTAL FRANCE (ARNAY LE DUC),1
1786,1787,Pt Bridgestone Tire Indonesia,Bridgestone Do Brasil Industria E Comenrcio Ltda,1
1798,1799,Brenntag Peru S.A.C.,Brenntag Group,1
1962,1963,"Carlisle Coatings & Waterproofing, Inc.",Carlisle Coatings & Wtrprfng,1
2054,2055,Pt Bridgestone Tire Indonesia,"Thai Bridgestone Co., Ltd.",1
2772,2773,Onduline Polska Sp. z o.o.,Onduline S.A.,1


In [ ]:
train_df[train_df.is_duplicate==0].tail(20)

,pair_id,name_1,name_2,is_duplicate
497799,497800,Smtc,Forward Plastic(Shenzhen) Ltd.,0
497800,497801,Automotive Performance Materials,Swisspor Polska Sp. z o.o.,0
497801,497802,Eoc Polymers India Private Ltd.,Imp. Express India Private Ltd.,0
497802,497803,"Shanghai Gaoxinke Plastics Co., Ltd.","Huangyan Xingtai Plastic Mould Co., Ltd.",0
497803,497804,Qingdao Yang Ming International Material Techn...,Llc Chemical Technologies,0
497804,497805,Societe Des Transports,Ho Sports Co. Inc.,0
497805,497806,D.S International,Titron Rubber Industries (M) Sdn. B,0
497806,497807,Savita Polymers,"T.A. Corporation Pvt., Ltd.",0
497807,497808,Exel Logistics Inc.,Jh Logistics Inc.,0
497808,497809,"Computime Electric (Shenzhen) Co., Ltd.",A S International,0


In [7]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("en"))
exclude = set(string.punctuation)

In [8]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [9]:
def preprocess_txt1(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i != ""]
    return spls

In [10]:
name_1_corpus = train_df.name_1
name_2_corpus = train_df.name_2

In [11]:
print(len(name_1_corpus))
name_1_corpus[600]

497819


'Aim International'

In [12]:
n_1_all=[]
for line in tqdm_notebook(name_1_corpus):
        spls = preprocess_txt1(line)
        n_1_all.append(spls)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/497819 [00:00<?, ?it/s]

In [13]:
print(len(n_1_all))
n_1_all[600]

497819


['aim', 'international']

In [14]:
n_2_all=[]
for line in tqdm_notebook(name_2_corpus):
        spls = preprocess_txt1(line)
        n_2_all.append(spls)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/497819 [00:00<?, ?it/s]

#Word2Vec, Fasttext, Annoy

In [15]:
all_names = n_1_all+n_2_all

In [16]:
# create models

modelW2V = Word2Vec(sentences=all_names, size=100, window=5, min_count=1)

modelFT = FastText(sentences=all_names, size=100, min_count=1, window=5, workers=8)

In [17]:
# craete Annoy indexes
index_w2v = annoy.AnnoyIndex(100 ,'angular') #using angular metric
index_ft = annoy.AnnoyIndex(100 ,'angular')


for i, sent in enumerate(all_names):

    res1 = [modelW2V.wv[w] for w in sent if w in modelW2V.wv]
    v_w2v = sum(res1) / len(res1) if res1 else np.zeros(100)

    res2 = [modelFT.wv[w] for w in sent if w in modelFT.wv]
    v_ft = sum(res2) / len(res2) if res2 else np.zeros(100)

    index_w2v.add_item(i, v_w2v)
    index_ft.add_item(i, v_ft)



index_w2v.build(10)
index_ft.build(10)


True

In [18]:
names_raw = train_df.name_1 + train_df.name_2

In [19]:
def get_response(tweet, index, model):
  tweet = preprocess_txt1(tweet)
  w_v  = [model.wv[w] for w in tweet if w in model.wv ]
  tweet_vector = sum(w_v) / len(w_v) if w_v else np.zeros(100)
  result = index.get_nns_by_vector(tweet_vector, 5) #5 nearest

  return [names_raw[i] for i in result] #[train_df['name_2'].iloc[i] for i in result]  


In [38]:
# choose random brand
df_dub = train_df[train_df.is_duplicate==1]
rand_i = random.randint(0, df_dub.shape[0])
test_brand = df_dub.name_1.iloc[rand_i]
print(f'исходная компания {test_brand}')
print('----------------')
print(f'соответствие из исходной таблицы {df_dub.name_2.iloc[rand_i]}')

исходная компания Mitsubishi Chemical Performance
----------------
соответствие из исходной таблицы Mitsubishi Chemical China Commerce Ltd.


In [39]:
# word2vec vectors
response = get_response(test_brand, index_w2v, modelW2V)
print(test_brand)
print('*'*50)
for line in response:
  
  print(line)
  print('-'*50)

Mitsubishi Chemical Performance
**************************************************
Mitsubishi Chemical PerformanceMitsuboshi Philippines Corp
--------------------------------------------------
Mitsubishi Chemical PerformanceDro Chemicals Inc.
--------------------------------------------------
Mitsubishi Chemical PerformanceAmbica Corp
--------------------------------------------------
Mitsubishi Chemical PerformanceMir International Co., Ltd.
--------------------------------------------------
Mitsubishi Chemical PerformanceLg Chem Ltd.
--------------------------------------------------


In [40]:
#FastText vectors
response = get_response(test_brand, index_ft, modelFT)
print(test_brand)
print('*'*50)
for line in response:
  
  print(line)
  print('-'*50)

Mitsubishi Chemical Performance
**************************************************
Mitsubishi Chemical PerformanceMitsuboshi Philippines Corp
--------------------------------------------------
Mitsubishi Chemical PerformanceDro Chemicals Inc.
--------------------------------------------------
Mitsubishi Chemical PerformanceAmbica Corp
--------------------------------------------------
Mitsubishi Chemical PerformanceMir International Co., Ltd.
--------------------------------------------------
Mitsubishi Chemical PerformanceLg Chem Ltd.
--------------------------------------------------


#Both models dive good result